# Práctica sobre algoritmos genéticos

<img src="imgs/tren.jpg">


Un área ferroviaria de carga/descarga con una única vía de entrada y otra salida se compone de tres muelles de carga/descarga: Op1, Op2 y Op3, correspondientes a contenedores, carbón y gas. Por tanto, cada tren que llega se dirige a un muelle en función de su carga. Un tren tarda en cargar o descargar un tiempo proporcional al número de vagones que arrastra. Cada día llegan secuencialmente n trenes. Si los trenes son de cargas distintas, pueden entrar en paralelo a los muelles. Cuando dos trenes con el mismo tipo de carga se encuentran seguidos, el segundo debe esperar por el primero, así como todos los demás que se encuentren por detrás.


Se nos plantea resolver, mediante un algoritmo genético, el problema de la ordenación en la entrada de los trenes para minimizar el tiempo de paso del conjunto de trenes.


<img src="imgs/docks.jpg" width="70%">

In [33]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

## Trenes

En el siguiente código, <code>random_trains_generation</code> genera los trenes que llegarán en un día. Cada ordenación diferente de este conjunto corresponderá con un **individuo** en nuestro eaquema de algoritmo genético.

In [34]:
import random
from deap import base, creator, tools

class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)


def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op, i))

    return train_list

In [35]:
trains = random_trains_generation(100)
def randomize_trains():
    new_trains = trains[:]
    random.shuffle(new_trains)
    return new_trains

#creamos una clase de tipo lista que implemente la funcion anterior y devuelva una lista de trenes aleatoria
class Trains_list(list):
    def __init__(self):
        super().__init__(randomize_trains())

In [36]:
creator.create("Fitness", base.Fitness, weights=(-1.0, )) 
creator.create("Individual", Trains_list, fitness=creator.Fitness)

In [37]:
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

La función de evalueción corregida

In [38]:
def evaltrains2(individual):
    time = 0
    operations = {"op1": 0, "op2": 0, "op3": 0}
    for trains in individual:
        wagons = trains.wagons
        if operations[trains.op] == 0: #si el muelle esta libre
            operations[trains.op] = wagons #pues esta en espare hasta que termine
        else:
            time_in = operations[trains.op] #el teimpo de espera del muelle que toque
            for k, v in operations.items():
                #no puede haber operaciones con valores menores a 0
                if v - time_in < 0: #si el tiempo espera es menor que el tiempo de espera del muelle, poner tiempo a 0, el tiempo no debería ser negativo
                    operations[k] = 0
                else:
                    operations[k] -= time_in #si no, restar el tiempo de espera del muelle
            time += time_in  #sumar el tiempo de espera del muelle
            operations[trains.op] = wagons #meter el tren en el muelle tras haber esperado
    time += max(operations.values()) #tardaremos el tiempo máximo entre todos los muelles
    return time,

In [39]:
def cxSet2(ind1, ind2):
    cx_index = np.random.randint(len(ind1))

    child1 = ind1[cx_index:]
    child2 = ind2[cx_index:]

    for x in ind2:
        if x not in child1:
            child1.append(x)

    for x in ind1:
        if x not in child2:
            child2.append(x)

    ind1[:] = child1
    ind2[:] = child2
    return ind1, ind2


In [40]:
def get_group(ind1, ind1_left, ind2_right):
    group = []
    for x in ind1:
        if x not in ind1_left and x not in ind2_right:
            group.append(x)
    return group

In [41]:
def cxSet3(ind1, ind2):
    cx_index = np.random.randint(len(ind1))

    ind1_right = ind1[cx_index:]
    ind2_right = ind2[cx_index:]
    ind1_left = ind1[:cx_index]
    ind2_left = ind2[:cx_index]

    #elementos de ind1 que no estan en su parte izquierda(primer trozo) ni en la parte a discriminar de ind2
    group1 = get_group(ind1, ind1_left, ind2_right)
    #elementos de ind2 que no estan en su parte izquierda(primer trozo) ni en la parte a discriminar de ind1
    group2 = get_group(ind2, ind2_left, ind1_right)
    #Creamos un nuevo lado derecho de ind1 e ind2 en donde no esten los trenes que ya estan en el trozo izquierdo que vamos a usar
    #y si no se cumple, rellenamos con un tren del individuo a tratar, los cuales si recogen las condiciones que queremos.
    new_ind1_right = []
    for x in ind2_right:
        if x not in ind1_left:
            new_ind1_right.append(x)
        else:
            new_ind1_right.append(group1.pop(0))

    new_ind2_right = []
    for x in ind1_right:
        if x not in ind2_left:
            new_ind2_right.append(x)
        else:
            new_ind2_right.append(group2.pop(0))

    ind1[:] = ind1_left + new_ind2_right
    ind2[:] = ind2_left + new_ind1_right
    return ind1, ind2

In [42]:
toolbox.register("evaluate", evaltrains2)
toolbox.register("mate", cxSet3)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [43]:
def main():
    pop = toolbox.population(n=50)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    #algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats, halloffame=hof)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
    return pop, stats, hof

In [44]:
pop, stats, hof = main()

gen	nevals	avg      	std          	min   	max    
0  	50    	[1125.26]	[51.64331903]	[999.]	[1228.]
1  	34    	[1078.18]	[39.13345883]	[999.]	[1152.]
2  	28    	[1044.1] 	[28.76612591]	[999.]	[1141.]
3  	22    	[1024.24]	[19.31482332]	[998.]	[1080.]
4  	22    	[1007.32]	[17.30715459]	[945.]	[1036.]
5  	36    	[1002.6] 	[19.43090322]	[960.]	[1070.]
6  	36    	[997.14] 	[19.3690578] 	[960.]	[1046.]
7  	28    	[988.12] 	[18.92262138]	[960.]	[1052.]
8  	36    	[984.62] 	[29.24235969]	[950.]	[1075.]
9  	26    	[975.82] 	[25.27978639]	[927.]	[1066.]
10 	24    	[965.88] 	[27.39827732]	[927.]	[1097.]
11 	28    	[958.72] 	[26.51719442]	[927.]	[1074.]
12 	37    	[951.38] 	[34.13613335]	[926.]	[1090.]
13 	27    	[944.02] 	[31.95089357]	[920.]	[1072.]
14 	25    	[939.08] 	[26.69744557]	[919.]	[1034.]
15 	32    	[940.02] 	[25.3672939] 	[919.]	[1018.]
16 	36    	[932.48] 	[16.86919085]	[919.]	[995.] 
17 	28    	[929.34] 	[16.53554958]	[919.]	[1007.]
18 	35    	[925.44] 	[13.09986259]	[919.]	[980.] 


In [45]:
#calculate the time of the initial trains list
time_of_operations = {"op1": 0, "op2": 0, "op3": 0}
for i in trains:
    time_of_operations[i.op] += i.wagons
print("el tiempo mínimo será: ",max(time_of_operations["op1"], time_of_operations["op2"], time_of_operations["op3"]))
print("el tiempo del algoritmo será: ",hof[0].fitness.values[0])


el tiempo mínimo será:  731
el tiempo del algoritmo será:  799.0


Miramos que no hayan trenes repetidos

In [46]:
licence_plates = []
for t in hof[0]:
    licence_plates.append(t.licence_plate)
    
print("Número de trenes: ", len(hof[0]))
print(len(licence_plates))
print(licence_plates)

Número de trenes:  100
100
[48, 14, 72, 71, 50, 33, 19, 18, 77, 94, 25, 73, 36, 68, 0, 81, 38, 91, 12, 31, 40, 57, 79, 69, 80, 54, 82, 59, 83, 4, 76, 22, 3, 74, 24, 35, 64, 67, 42, 29, 39, 55, 8, 47, 70, 84, 44, 78, 99, 21, 13, 98, 7, 90, 26, 6, 49, 65, 52, 1, 46, 87, 20, 5, 15, 10, 88, 56, 96, 34, 41, 28, 62, 9, 17, 27, 30, 89, 86, 93, 45, 75, 2, 92, 97, 37, 53, 16, 11, 58, 51, 61, 66, 32, 85, 63, 60, 43, 23, 95]


Miramos cual es la configuración de trenes que menos tiempo tarda en pasar

In [47]:
licencias = []
for i in hof[0]:
    licencias.append(i.licence_plate)
print(sorted(licencias))
print(len(licencias))
print(len(set(licencias)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
100
100
